In [1]:
import sys
sys.path.append('../..')
sys.path.append('../')
from datasets import load_dataset, load_dataset_builder
import lightning as L

from astropile.utils import cross_match_datasets
from photo_z_data import PhotoZWrapper
from photo_z_model import SimpleCNN
from utils import split_dataset

/mnt/sw/nix/store/b4q5asj8flwlgmaijgj1r6wbmnls5x8k-python-3.9.15-view/lib/python3.9/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: /mnt/sw/nix/store/7bjp92vyxq4vmcm6q2kwdv325d50n362-py-torchvision-0.14.1/lib/python3.9/site-packages/torchvision/image.so: undefined symbol: _ZN5torch3jit17parseSchemaOrNameERKNSt7__cxx1112basic_stringIcSt11char_traitsIcESaIcEEE
  warn(f"Failed to load image Python extension: {e}")


In [2]:
# Load Dataset Builders 
hsc_builder = load_dataset_builder('/mnt/ceph/users/polymathic/AstroPile_tiny/hsc/hsc.py', trust_remote_code=True)
desi_builder = load_dataset_builder('/mnt/ceph/users/polymathic/AstroPile_tiny/desi/desi.py', trust_remote_code=True)

# Cross-Match Datasets with AstroPile
hsc_meets_desi = cross_match_datasets(desi_builder, hsc_builder,
                                      matching_radius=1.0,
                                      keep_in_memory=True,
                                      )
hsc_meets_desi.set_format('torch')

/mnt/home/lparker/python_envs/astrokernel/lib/python3.9/site-packages/huggingface_hub/repocard.py:105: UserWarning: Repo card metadata block was not found. Setting CardData to empty.
  warnings.warn("Repo card metadata block was not found. Setting CardData to empty.")


Initial number of matches:  1286
Number of matches lost at healpix region borders:  0
Final size of cross-matched catalog:  1286


In [3]:
# Use prebuilt split function to split the dataset (currently supports naive)
train_dataset, test_dataset = split_dataset(hsc_meets_desi, split='naive')

In [4]:
# Create PhotoZWrapper for training and testing
photo_z = PhotoZWrapper(
    train_dataset,
    test_dataset, 
    feature_flag='image.array', # feature flag
    label_flag='Z',             # label flag
    feature_dynamic_range=True,
    label_dynamic_range=False,
    feature_z_score=True,
    label_z_score=True,
    loading='iterated',         # iterated or full
    batch_size=128, 
    num_workers=16, 
    val_size=0.1, 
    )    

# Create SimpleCNN model
model = SimpleCNN(input_channels=5, layer_width=64, num_layers=3, num_classes=1, learning_rate=5e-3)

In [5]:
import lightning as L
from lightning.pytorch.callbacks import TQDMProgressBar

trainer = L.Trainer(max_epochs=20, accelerator='gpu', logger=True)
trainer.fit(model=model, datamodule=photo_z)

/mnt/home/lparker/python_envs/astrokernel/lib/python3.9/site-packages/lightning/fabric/plugins/environments/slurm.py:204: The `srun` command is available on your system but is not used. HINT: If your intention is to run Lightning on SLURM, prepend your python command with `srun` like so: srun python /mnt/sw/nix/store/b4q5asj8flwlgmaijgj1r6wbmnls5x8k-p ...
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/mnt/home/lparker/python_envs/astrokernel/lib/python3.9/site-packages/lightning/fabric/plugins/environments/slurm.py:204: The `srun` command is available on your system but is not used. HINT: If your intention is to run Lightning on SLURM, prepend your python command with `srun` like so: srun python /mnt/sw/nix/store/b4q5asj8flwlgmaijgj1r6wbmnls5x8k-p ...
You are using a CUDA device ('NVIDIA RTX A6000') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

/mnt/home/lparker/python_envs/astrokernel/lib/python3.9/site-packages/lightning/pytorch/loops/fit_loop.py:298: The number of training batches (8) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.


Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

In [9]:
trainer.test(datamodule=photo_z)

/mnt/home/lparker/python_envs/astrokernel/lib/python3.9/site-packages/lightning/pytorch/trainer/connectors/checkpoint_connector.py:145: `.test(ckpt_path=None)` was called without a model. The best model of the previous `fit` call will be used. You can pass `.test(ckpt_path='best')` to use the best model or `.test(ckpt_path='last')` to use the last model. If you pass a value, this warning will be silenced.
Restoring states from the checkpoint path at /mnt/home/lparker/Documents/AstroFoundationModel/AstroPile_prototype/baselines/photo_z/lightning_logs/version_5/checkpoints/epoch=19-step=160.ckpt
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
Loaded model weights from the checkpoint at /mnt/home/lparker/Documents/AstroFoundationModel/AstroPile_prototype/baselines/photo_z/lightning_logs/version_5/checkpoints/epoch=19-step=160.ckpt


Testing: |          | 0/? [00:00<?, ?it/s]

────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
        test_loss           0.5690107941627502
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


[{'test_loss': 0.5690107941627502}]